<a href="https://colab.research.google.com/github/Broccolihua/net_work_test/blob/main/test_net_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding:utf-8- -*-
"""
@Time  :   2022/9/29 17:17
@Author : HYS
@FileName : train.py
"""
import torch
import torchvision
from torch import nn
from torch.nn import Sequential
from torch.utils.tensorboard import SummaryWriter
import time


# from model import *
#准备数据
from torch.utils.data import DataLoader

#定义训练的设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



train_data = torchvision.datasets.CIFAR10("../dataset", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10("../dataset", train=False, transform=torchvision.transforms.ToTensor(),

                                         download=True)

#length  长度
train_data_size = len(train_data)
test_data_size = len(test_data)

#如果train_data_size = 10 ，训练数据集长度为 ：10
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))


#利用DataLoader 来加载数据集

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

#创建网络模型
class Hys(nn.Module):
    def __init__(self):
        super(Hys, self).__init__()
        self.model = Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10),
        )

    def forward(self, x):
        x = self.model(x)
        return x
hys = Hys()
hys = hys.to(device)
# if torch.cuda.is_available():    #判断是否有gpu  有就可以 调用到gpu上
#     hys = hys.cuda()


#损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
# loss_fn = loss_fn.cuda()

#优化器
#1e-2=1*(10)^(-2) = 1/100   =0.01
learning_rate = 1e-2
optimizer = torch.optim.SGD(hys.parameters(), lr=learning_rate)

#设置训练网络的一些参数
#记录训练的次数
total_train_step = 0
#记录测试的次数
total_test_step = 0
#设置训练的轮数
epoch = 30

#添加 tensorboard2
writer = SummaryWriter("../log_train")
start_time = time.time()            # 开始时间
for i in range(epoch):
    print("-----------第 {} 轮训练开始-----------".format(i+1))

    #训练开始步骤
    hys.train()   ##只对Dropout  BatchNorm etc 有作用
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        # imgs = imgs.cuda()                        #  数据转移到cuda
        # targets = targets.cuda()                  #  标记 转移到cuda
        outputs = hys(imgs)
        loss = loss_fn(outputs, targets)

        #优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            end_time = time.time()              # 结束时间
            print(end_time - start_time)
            print("训练次数： {}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    #测试步骤开始
    hys.eval()  #只对Dropout  BatchNorm etc 有作用
    total_test_loss = 0
    total_accuracy = 0 #继续优化
    with torch.no_grad():
        for dat in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            # imgs = imgs.cuda()         #  数据转移到cuda
            # targets = targets.cuda()            #  标记 转移到cuda
            outputs = hys(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy
    print("整体测试集上的Loss： {}".format(total_test_loss))
    print("整体测试集上的正确率：{}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step = total_train_step + 1

    torch.save(hys,"hys_{}.pth".format(i))
    #torch.save(hys.state_dict(),"hys_gpu_{}。pth".format(i))    官方推荐
    print("模型已保存")

writer.close()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../dataset/cifar-10-python.tar.gz to ../dataset
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
-----------第 1 轮训练开始-----------
9.718226671218872
训练次数： 100, Loss: 2.2784738540649414
10.630531787872314
训练次数： 200, Loss: 2.275907039642334
11.566973209381104
训练次数： 300, Loss: 2.2360167503356934
12.47831106185913
训练次数： 400, Loss: 2.130283832550049
13.37966775894165
训练次数： 500, Loss: 2.0297558307647705
14.28174376487732
训练次数： 600, Loss: 1.99735689163208
15.183218955993652
训练次数： 700, Loss: 2.0309369564056396
整体测试集上的Loss： 320.46382117271423
整体测试集上的正确率：0.07849999517202377
模型已保存
-----------第 2 轮训练开始-----------
17.396642446517944
训练次数： 800, Loss: 1.8786262273788452
18.680456161499023
训练次数： 900, Loss: 1.8351472616195679
19.77005887031555
训练次数： 1000, Loss: 1.9039226770401
20.67871117591858
训练次数： 1100, Loss: 1.9433749914169312
21.581816911697388
训练次数： 1200, Loss: 1.7156836986541748
22.68028497695923
训练次数： 1300, Loss: 1.686324954032898
23.599256992340088
训练次数： 1400, Los